In [1]:
#from curl_cffi.requests import Session
# import yfinance as yf
import pandas as pd
import numpy as np
# import ffn
#from langchain_core.tools import tool
#from duckduckgo_search import DDGS
from openai import OpenAI
import os
import html
import urllib.parse
import requests
from bs4 import BeautifulSoup
#from googlesearch import search  # google 라이브러리 사용
import requests
from bs4 import BeautifulSoup
import os
#from pytrends.request import TrendReq
#from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import contextlib
import requests
from urllib3.exceptions import InsecureRequestWarning

old_merge_environment_settings = requests.Session.merge_environment_settings
@contextlib.contextmanager
def no_ssl_verification():
    opened_adapters = set()

    def merge_environment_settings(self, url, proxies, stream, verify, cert):
        # Verification happens only once per connection so we need to close
        # all the opened adapters once we're done. Otherwise, the effects of
        # verify=False persist beyond the end of this context manager.
        opened_adapters.add(self.get_adapter(url))

        settings = old_merge_environment_settings(self, url, proxies, stream, verify, cert)
        settings['verify'] = False

        return settings

    requests.Session.merge_environment_settings = merge_environment_settings

    try:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', InsecureRequestWarning)
            yield
    finally:
        requests.Session.merge_environment_settings = old_merge_environment_settings

        for adapter in opened_adapters:
            try:
                adapter.close()
            except:
                pass

from transformers import pipeline
with no_ssl_verification():
    pipe = pipeline("text-classification", model="snunlp/KR-FinBert-SC")

c:\python_work\gpt_agent_2025\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


In [2]:
load_dotenv()
client_id = os.getenv("NAVER_CLIENT_ID")
client_secret = os.getenv("NAVER_CLIENT_SECRET")

In [3]:
import os
import requests
import urllib.parse
import html
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
client_id = os.getenv("NAVER_CLIENT_ID")
client_secret = os.getenv("NAVER_CLIENT_SECRET")

In [5]:
import os
import requests
import urllib.parse
import html
from bs4 import BeautifulSoup
from datetime import datetime

def get_news_list(company_name: str, display=10) -> list:
    client_id = os.getenv("NAVER_CLIENT_ID")  # 환경 변수에서 클라이언트 ID 가져오기
    client_secret = os.getenv("NAVER_CLIENT_SECRET")  # 환경 변수에서 클라이언트 Secret 가져오기
    
    # 쿼리 URL 인코딩
    query = urllib.parse.quote(company_name)
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={display}&sort=date"
    
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
        return []
    
    print("API 호출 성공!")
    items = response.json().get("items", [])
    
    if not items:
        print(f"❗ '{company_name}' 관련 뉴스가 없습니다.")
        return []

    # 뉴스 기사 내용 정제 함수
    def clean(text):
        return html.unescape(BeautifulSoup(text, "html.parser").get_text())

    news_items = []
    
    for item in items:
        title = clean(item.get("title", "제목 없음"))
        desc = clean(item.get("description", ""))
        link = item.get("originallink") or item.get("link") or f"https://search.naver.com/search.naver?query={query}"
        pub_date = item.get("pubDate", "날짜 없음")  # 기사 발행 날짜
        
        # 발행 날짜 형식 변환
        try:
            pub_date = datetime.strptime(pub_date, '%a, %d %b %Y %H:%M:%S %z').strftime('%Y-%m-%d %H:%M:%S')
        except Exception as e:
            pub_date = "날짜 변환 오류"  # 날짜 형식 오류 시 기본 메시지
            
        # 결과에 제목, 링크, 설명, 날짜 추가
        news_items.append({
            "제목": title,
            "내용": desc,
            "링크": link,
            "발행 날짜": pub_date  # 날짜 형식 추가
        })
    
    return news_items


In [6]:

# 실행 예시
company_name = "팔란티어"  # 검색할 기업 이름
with no_ssl_verification():
    news = get_news_list(company_name, display= 20)

# 결과 출력
if news:
    for idx, article in enumerate(news, 1):
        print(f"{idx}. 제목: {article['제목']}")
        print(f"   내용: {article['내용']}")
        print(f"   링크: {article['링크']}")

API 호출 성공!
1. 제목: 팔란티어 주가 8월 1일 하락 마감, AI 4대 플랫폼과 주가 동향 분석
   내용: 팔란티어 테크놀로지스(Palantir Technologies Inc)가 지난 1일 기준 나스닥 시장에서 하락세를 기록했다.... 출처 : 네이버페이 증권 팔란티어의 시가총액은 약 3,641억 달러(한화 약 505조 원)로 집계되었으며, 지난... 
   링크: https://www.job-post.co.kr/news/articleView.html?idxno=166570
2. 제목: [주간증시전망] 세제 쇼크·실적 변수 겹친 코스피···변동성 확대
   내용: AMD, 팔란티어 등 AI 하드웨어, 소프트웨어 업체의 실적을 통해 그 모멘텀을 이어갈 수 있는지가 관건"이라고 말했다. 한 연구원은 "국내에서도 현대로템, LIG넥스원 등 방산주, NAVER, 카카오 등 인터넷주, 에코프로비엠 등... 
   링크: https://www.seoulfn.com/news/articleView.html?idxno=602812
3. 제목: [뉴욕 주간 프리뷰] ②'배드이즈배드' 국면 전환? 평온함에 균열
   내용: 한편 S&P500 기업의 60%가 분기 결산을 공개한 가운데 이번 주에는 팔란티어테크놀로지스(종목코드: PLTR, 4일)과 AMD(종목코드 동일, 5일), 캐터필러(CAT, 5일) 월트디즈니(DIS, 6일)가 실적을 내놓는다. 주식시장의 AI 투자... 
   링크: https://www.newspim.com/news/view/20250804000288
4. 제목: 꿈에서 깬 월가…경기 침체 경고음에 선물↓
   내용: 팔란티어 테크놀로지스는 미 육군과 100억 달러 규모의 소프트웨어 및 데이터 계약을 체결한 후 4일에 실적을 보고한다. 칩 대기업인 AMD는 5일에 실적을 보고할 예정이며, 이는 27일에야 발표될 엔비디아 실적에 간단한... 
   링크: https://www.fortunekorea.co.kr/news/articleView.html?

In [7]:
news_df = pd.DataFrame(news)

In [8]:
news_df

,제목,내용,링크,발행 날짜
0,"팔란티어 주가 8월 1일 하락 마감, AI 4대 플랫폼과 주가 동향 분석",팔란티어 테크놀로지스(Palantir Technologies Inc)가 지난 1일 ...,https://www.job-post.co.kr/news/articleView.ht...,2025-08-04 10:28:00
1,[주간증시전망] 세제 쇼크·실적 변수 겹친 코스피···변동성 확대,"AMD, 팔란티어 등 AI 하드웨어, 소프트웨어 업체의 실적을 통해 그 모멘텀을 이...",https://www.seoulfn.com/news/articleView.html?...,2025-08-04 10:27:00
2,[뉴욕 주간 프리뷰] ②'배드이즈배드' 국면 전환? 평온함에 균열,한편 S&P500 기업의 60%가 분기 결산을 공개한 가운데 이번 주에는 팔란티어테...,https://www.newspim.com/news/view/20250804000288,2025-08-04 10:20:00
3,꿈에서 깬 월가…경기 침체 경고음에 선물↓,팔란티어 테크놀로지스는 미 육군과 100억 달러 규모의 소프트웨어 및 데이터 계약을...,https://www.fortunekorea.co.kr/news/articleVie...,2025-08-04 10:10:00
4,[주간 뉴욕증시] 고용둔화에 불안 커진 증시…실적 시즌 변동성 확대,"기업 실적 발표로는 히임스앤허스, 팔란티어, 타이슨푸드 등이 예정돼 있다. 5일에는...",https://www.newsis.com/view/NISX20250804_00032...,2025-08-04 10:10:00
5,트럼프發 무역협상 언제까지…팔란티어·AMD 실적 주목 [글로벌마켓레이...,"이번 주에는 팔란티어 테크놀로지(4일), 어드밴스드 마이크로 디바이시스(AMD, 5...",https://biz.heraldcorp.com/article/10546060?re...,2025-08-04 09:54:00
6,"팔란티어, 8월 1일 154.27달러 마감, 2.58% 하락","미국 동부시간 1일 기준, 네이버페이 증권에 따르면, 팔란티어 테크놀로지스의 주가는...",https://www.topstarnews.net/news/articleView.h...,2025-08-04 09:42:00
7,"코스피,'세제 개편 불확실성' 속 보합…변동성 확대 전망","한지영 키움증권 연구원은 ""금요일 미국 고용 쇼크 여진, 미국 7월 ISM 서비스업...",https://view.asiae.co.kr/article/2025080409395...,2025-08-04 09:39:00
8,"[개장시황] 코스피, 장 초반 보합세…'세제개편안' 후유증 여전","한지영 키움증권 연구원은 ""이번주 한국 증시는 금요일 발표된 미국 고용지표 충격의 ...",https://www.newspim.com/news/view/20250804000148,2025-08-04 09:29:00
9,[뉴욕 마켓 브리핑](4일),"6개월 ▲기업 실적발표: 팔란티어 테크놀로지, 로우스, 온 세미컨덕터, 버텍스 파마...",https://news.einfomax.co.kr/news/articleView.h...,2025-08-04 09:28:00


In [9]:

# 4. 감성 분석을 통해 점수 부여
def analyze_sentiment(text):
    result = pipe(text)  # 모델로 텍스트 분석
    label = result[0]['label']  # '긍정', '부정', '중립' 중 하나
    score = result[0]['score']  # 감성 점수
    return label, score

# 5. 감성 분석 결과를 데이터프레임에 추가
news_df[['감성', '감성 점수']] = news_df['내용'].apply(lambda x: pd.Series(analyze_sentiment(x)))

c:\python_work\gpt_agent_2025\myenv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [10]:
news_df

,제목,내용,링크,발행 날짜,감성,감성 점수
0,"팔란티어 주가 8월 1일 하락 마감, AI 4대 플랫폼과 주가 동향 분석",팔란티어 테크놀로지스(Palantir Technologies Inc)가 지난 1일 ...,https://www.job-post.co.kr/news/articleView.ht...,2025-08-04 10:28:00,negative,0.997853
1,[주간증시전망] 세제 쇼크·실적 변수 겹친 코스피···변동성 확대,"AMD, 팔란티어 등 AI 하드웨어, 소프트웨어 업체의 실적을 통해 그 모멘텀을 이...",https://www.seoulfn.com/news/articleView.html?...,2025-08-04 10:27:00,positive,0.982928
2,[뉴욕 주간 프리뷰] ②'배드이즈배드' 국면 전환? 평온함에 균열,한편 S&P500 기업의 60%가 분기 결산을 공개한 가운데 이번 주에는 팔란티어테...,https://www.newspim.com/news/view/20250804000288,2025-08-04 10:20:00,neutral,0.927174
3,꿈에서 깬 월가…경기 침체 경고음에 선물↓,팔란티어 테크놀로지스는 미 육군과 100억 달러 규모의 소프트웨어 및 데이터 계약을...,https://www.fortunekorea.co.kr/news/articleVie...,2025-08-04 10:10:00,positive,0.997782
4,[주간 뉴욕증시] 고용둔화에 불안 커진 증시…실적 시즌 변동성 확대,"기업 실적 발표로는 히임스앤허스, 팔란티어, 타이슨푸드 등이 예정돼 있다. 5일에는...",https://www.newsis.com/view/NISX20250804_00032...,2025-08-04 10:10:00,neutral,0.999785
5,트럼프發 무역협상 언제까지…팔란티어·AMD 실적 주목 [글로벌마켓레이...,"이번 주에는 팔란티어 테크놀로지(4일), 어드밴스드 마이크로 디바이시스(AMD, 5...",https://biz.heraldcorp.com/article/10546060?re...,2025-08-04 09:54:00,neutral,0.952082
6,"팔란티어, 8월 1일 154.27달러 마감, 2.58% 하락","미국 동부시간 1일 기준, 네이버페이 증권에 따르면, 팔란티어 테크놀로지스의 주가는...",https://www.topstarnews.net/news/articleView.h...,2025-08-04 09:42:00,negative,0.956695
7,"코스피,'세제 개편 불확실성' 속 보합…변동성 확대 전망","한지영 키움증권 연구원은 ""금요일 미국 고용 쇼크 여진, 미국 7월 ISM 서비스업...",https://view.asiae.co.kr/article/2025080409395...,2025-08-04 09:39:00,negative,0.771603
8,"[개장시황] 코스피, 장 초반 보합세…'세제개편안' 후유증 여전","한지영 키움증권 연구원은 ""이번주 한국 증시는 금요일 발표된 미국 고용지표 충격의 ...",https://www.newspim.com/news/view/20250804000148,2025-08-04 09:29:00,neutral,0.998445
9,[뉴욕 마켓 브리핑](4일),"6개월 ▲기업 실적발표: 팔란티어 테크놀로지, 로우스, 온 세미컨덕터, 버텍스 파마...",https://news.einfomax.co.kr/news/articleView.h...,2025-08-04 09:28:00,neutral,0.999796
